In [1]:
%load_ext autoreload
%autoreload 2
# %matplotlib widget

In [2]:
import xarray as xr
import numpy as np
import pandas as pd

import ecephys_analyses as ea 

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

In [3]:
xr.set_options(keep_attrs=True)

In [16]:
def summarize(subject, condition, spws, norm_spws):
    return dict(subject=subject, condition=condition, amplitude=spws.sink_amplitude.mean() / norm_spws.sink_amplitude.mean(), integral=spws.sink_integral.mean() / norm_spws.sink_integral.mean(), duration=spws.duration.mean() / norm_spws.duration.mean())

In [17]:
subjects = ["Segundo", "Valentino", "Doppio", "Alessandro", "Allan"]
subject_dfs = list()
for subject in subjects:
    norm_hyp, norm_spws = ea.load_baseline_light_period_nrem(subject)
    dep_hyp, dep_spws = ea.load_deprivation(subject)
    early_hyp, early_spws = ea.load_early_recovery_nrem(subject)
    late_hyp, late_spws = ea.load_late_recovery_nrem(subject)
    rec_2h_hyp, rec_2h_spws = ea.load_first2h_recovery_nrem(subject)
    rec_6h_hyp, rec_6h_spws = ea.load_recovery_nrem(subject)
    bsl_2h_hyp, bsl_2h_spws = ea.load_first2h_recovery_match_nrem(subject, rec_2h_hyp)

    subject_df = pd.DataFrame.from_dict(
        [
            summarize(subject, "baseline-light-period-nrem", norm_spws, norm_spws),
            summarize(subject, "recovery-first2h-match", bsl_2h_spws, norm_spws),
            summarize(subject, "recovery-first2h", rec_2h_spws, norm_spws),
            summarize(subject, "recovery-first1h", early_spws, norm_spws),
            summarize(subject, "recovery-last1h", late_spws, norm_spws),
            summarize(subject, "recovery-full", rec_6h_spws, norm_spws),
            summarize(subject, "deprivation", dep_spws, norm_spws)
        ], 
    )

    subject_dfs.append(subject_df)

In [18]:
df = pd.concat(subject_dfs)
df

,subject,condition,amplitude,integral,duration
0,Segundo,baseline-light-period-nrem,1.000000,1.000000,1.000000
1,Segundo,recovery-first2h-match,1.005579,0.999614,0.995105
2,Segundo,recovery-first2h,1.057677,1.066537,1.018321
3,Segundo,recovery-first1h,1.069748,1.062225,1.002322
4,Segundo,recovery-last1h,1.038233,1.013597,0.986130
5,Segundo,recovery-full,1.051670,1.052216,1.011139
6,Segundo,deprivation,1.050451,0.888992,0.837936
0,Valentino,baseline-light-period-nrem,1.000000,1.000000,1.000000
1,Valentino,recovery-first2h-match,1.014799,1.006638,0.992400
2,Valentino,recovery-first2h,1.068401,1.119120,1.067167


In [19]:
df.to_csv('mean-spw-properties.csv', index=False)